# DMA Crossover + Two-Green Entry / Two-Red Exit — v2 (No Plots by Default)
Adds **optional filters** to reduce false/negative trades and disables plotting by default.
Pure Python (`pandas`, `yfinance`, `matplotlib` only if you turn plots back on).

### Optional Filters You Can Toggle
- Price above 200DMA (avoid weak regimes)
- 20DMA slope positive (avoid flat/whipsaw)
- RSI filter (e.g., RSI>50)
- ADX filter (e.g., ADX>18–25 for trend strength)
- Volume surge vs. 20-day average
- ATR band: skip too-low or too-high volatility regimes
- Protective exit: close below 20DMA exits at next open (prevents waiting for two red candles)


## Parameters (edit me)

In [49]:

# ====================== USER PARAMETERS ======================
TICKERS = [
        "ATUL.NS",
    "CENTRALBK.NS",
    "COCHINSHIP.NS"
    "FACT.NS",
    "GSPL.NS",
    "HINDPETRO.NS",
    "IRFC.NS",
    "LUPIN.NS",
    "NBCC.NS",
    "TARIL.NS",
    "UCOBANK.NS"
]

START_DATE = "2025-01-01"
END_DATE   = None

FAST_DMA = 10
SLOW_DMA = 20
REGIME_DMA = 200  # for regime filter

# Entry timing relative to cross
REQUIRE_CROSS_EXACTLY_T_MINUS_2 = True
CROSS_WINDOW_AFTER = 3  # used only if above flag is False

TZ = "Asia/Kolkata"

# Backtest settings
INITIAL_CAPITAL = 50000.0
RISK_PER_TRADE = 1.0
SLIPPAGE_BPS = 5
FEES_BPS = 2

# -------- Optional Filters (set True/False) --------
USE_FILTER_PRICE_ABOVE_200DMA = False
USE_FILTER_SLOW_SLOPE_POSITIVE = False          # slope of 20DMA positive
USE_FILTER_RSI = False; RSI_LEN = 14; RSI_MIN = 50
USE_FILTER_ADX = False; ADX_LEN = 14; ADX_MIN = 18
USE_FILTER_VOLUME_SURGE = False; VOL_LOOKBACK = 20; VOL_FACTOR = 1.2
USE_FILTER_ATR_BAND = False; ATR_LEN = 14; ATR_MIN_PCT = 0.5; ATR_MAX_PCT = 4.0
    # Require ATR% (ATR/Close*100) to be within [ATR_MIN_PCT, ATR_MAX_PCT]


# Protective exit (optional)
PROTECTIVE_EXIT_ON_CLOSE_BELOW_20DMA = False

# Plot settings (disabled by default)
PLOT_LAST_N = 0        # 0 means no plotting
SAVE_PLOTS = False
PLOTS_DIR = "plots_dma_two_green_v2"
# ============================================================


## Install & Imports

In [50]:

# !pip install yfinance pandas numpy matplotlib

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import os

if SAVE_PLOTS and not os.path.isdir(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)


## Helpers: Indicators (RSI/ATR/ADX), Data, Signals, Backtest

In [51]:

def _to_tz_local_naive(df, tz):
    if df.index.tz is None:
        df.index = df.index.tz_localize('UTC').tz_convert(tz).tz_localize(None)
    else:
        df.index = df.index.tz_convert(tz).tz_localize(None)
    return df

def fetch_1d(ticker, start, end, tz):
    df = yf.download(ticker, start=start, end=end, interval='1d', auto_adjust=False, progress=False, multi_level_index=False)
    if df.empty: return df
    df = df.rename(columns=str.title)
    df = _to_tz_local_naive(df, tz).dropna(subset=['Open','High','Low','Close','Volume'])
    return df

def SMA(s, n):
    return s.rolling(n, min_periods=n).mean()

def RSI(close, n=14):
    delta = close.diff()
    gain = delta.clip(lower=0.0)
    loss = -delta.clip(upper=0.0)
    # Wilder's smoothing
    avg_gain = gain.ewm(alpha=1/n, adjust=False).mean()
    avg_loss = loss.ewm(alpha=1/n, adjust=False).mean()
    rs = avg_gain / avg_loss.replace(0, np.nan)
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(50)  # neutral when undefined

def ATR(high, low, close, n=14):
    prev_close = close.shift(1)
    tr = pd.concat([
        (high - low),
        (high - prev_close).abs(),
        (low - prev_close).abs()
    ], axis=1).max(axis=1)
    return tr.ewm(alpha=1/n, adjust=False).mean()

def ADX(high, low, close, n=14):
    # Wilder's ADX
    up_move = high.diff()
    down_move = -low.diff()
    plus_dm = np.where((up_move > down_move) & (up_move > 0), up_move, 0.0)
    minus_dm = np.where((down_move > up_move) & (down_move > 0), down_move, 0.0)
    tr = pd.concat([
        (high - low),
        (high - close.shift(1)).abs(),
        (low - close.shift(1)).abs()
    ], axis=1).max(axis=1)
    atr = tr.ewm(alpha=1/n, adjust=False).mean()
    plus_di = 100 * pd.Series(plus_dm, index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    minus_di = 100 * pd.Series(minus_dm, index=close.index).ewm(alpha=1/n, adjust=False).mean() / atr
    dx = ( (plus_di - minus_di).abs() / (plus_di + minus_di).replace(0,np.nan) ) * 100
    adx = dx.ewm(alpha=1/n, adjust=False).mean()
    return adx

def add_indicators(df, fast=10, slow=20, regime=200, rsi_len=14, adx_len=14, atr_len=14):
    o = df.copy()
    o[f'DMA_{fast}'] = SMA(o['Close'], fast)
    o[f'DMA_{slow}'] = SMA(o['Close'], slow)
    o[f'DMA_{regime}'] = SMA(o['Close'], regime)
    o['Green'] = o['Close'] > o['Open']
    o['Red']   = o['Close'] < o['Open']
    o['RSI'] = RSI(o['Close'], rsi_len)
    o['ATR'] = ATR(o['High'], o['Low'], o['Close'], atr_len)
    o['ATR_PCT'] = o['ATR'] / o['Close'] * 100
    o['ADX'] = ADX(o['High'], o['Low'], o['Close'], adx_len)
    # Slope of slow DMA (first difference)
    o['SLOW_SLOPE'] = o[f'DMA_{slow}'].diff()
    o['VOL_MA'] = SMA(o['Volume'], 20)
    return o

def mark_crosses(out, fast=10, slow=20):
    f, s = f'DMA_{fast}', f'DMA_{slow}'
    out['CrossUp'] = (out[f].shift(1) <= out[s].shift(1)) & (out[f] > out[s])
    out['CrossDn'] = (out[f].shift(1) >= out[s].shift(1)) & (out[f] < out[s])
    return out

def entry_signal(o, fast=10, slow=20,
                 exact_t_minus_2=True, cross_window_after=3,
                 regime_ok=True, slope_ok=True, rsi_ok=True, rsi_min=50,
                 adx_ok=True, adx_min=18, vol_surge=False, vol_lb=20, vol_factor=1.2,
                 atr_band=True, atr_min_pct=0.5, atr_max_pct=4.0):
    green_seq = (o['Green'] & o['Green'].shift(1))  # two greens ending at t
    higher_close = o['Close'] > o['Close'].shift(1)
    base = (green_seq & higher_close)

    # Cross timing
    if exact_t_minus_2:
        cross_cond = o['CrossUp'].shift(2).fillna(False)
    else:
        c = False
        for k in range(2, 2 + cross_window_after):
            c = c | o['CrossUp'].shift(k).fillna(False)
        cross_cond = c

    cond = base & cross_cond

    # Optional filters
    if regime_ok:
        cond &= o['Close'] > o[f'DMA_{200}']
    if slope_ok:
        cond &= o['SLOW_SLOPE'] > 0
    if rsi_ok:
        cond &= o['RSI'] >= rsi_min
    if adx_ok:
        cond &= o['ADX'] >= adx_min
    if vol_surge:
        cond &= o['Volume'] >= (o['VOL_MA'] * vol_factor)
    if atr_band:
        cond &= (o['ATR_PCT'] >= atr_min_pct) & (o['ATR_PCT'] <= atr_max_pct)

    return cond.fillna(False)

def exit_signal(o, slow=20, protective_on_close_below_slow=True):
    red_seq = (o['Red'] & o['Red'].shift(1))
    below_slow = o['Close'] < o[f'DMA_{slow}']
    base_exit = (red_seq & below_slow)
    if protective_on_close_below_slow:
        base_exit = base_exit | below_slow  # any close below 20DMA triggers exit next open
    return base_exit.fillna(False)

def backtest_long_only(df, enter_sig, exit_sig, initial_capital=100000.0, risk_frac=1.0,
                       slippage_bps=5, fees_bps=2):
    ohlc = df[['Open','High','Low','Close','Volume']].copy()
    open_next = ohlc['Open'].shift(-1)
    slippage = slippage_bps / 10000.0
    fees = fees_bps / 10000.0

    in_pos=False; qty=0; entry_price=np.nan; cash=initial_capital
    equity_curve=[]; trades=[]

    for i, (ts,row) in enumerate(ohlc.iterrows()):
        equity = cash + qty * row['Close']
        equity_curve.append((ts,equity))
        if i == len(ohlc)-1: continue

        if (not in_pos) and enter_sig.loc[ts]:
            px = open_next.loc[ts]
            if pd.isna(px) or px<=0: continue
            fill = px * (1+slippage)
            fee_amt = fill * fees
            alloc = equity * risk_frac
            qty = int(alloc // fill)
            if qty<=0: continue
            cash -= qty*fill + fee_amt
            entry_price = fill; in_pos=True
            trades.append({'Time': ohlc.index[i+1], 'Ticker': df.attrs.get('Ticker','N/A'),
                           'Action':'BUY','Qty':qty,'Price':fill,'Fees':fee_amt})
        elif in_pos and exit_sig.loc[ts]:
            px = open_next.loc[ts]
            if pd.isna(px) or px<=0: continue
            fill = px * (1-slippage)
            notional = qty*fill
            fee_amt = notional * fees
            cash += notional - fee_amt
            trades.append({'Time': ohlc.index[i+1], 'Ticker': df.attrs.get('Ticker','N/A'),
                           'Action':'SELL','Qty':qty,'Price':fill,'Fees':fee_amt,
                           'PnL': (fill-entry_price)*qty - fee_amt})
            qty=0; entry_price=np.nan; in_pos=False

    if len(ohlc):
        last_ts=ohlc.index[-1]
        equity_curve.append((last_ts, cash + qty*ohlc['Close'].iloc[-1]))
    ec = pd.DataFrame(equity_curve, columns=['Time','Equity']).set_index('Time')
    tr = pd.DataFrame(trades)
    return tr, ec


## Run for All Tickers

In [52]:

all_trades=[]; perf_rows=[]; all_ec={}
for t in TICKERS:
    print(f"Processing {t}...")
    df = fetch_1d(t, START_DATE, END_DATE, TZ)
    if df.empty:
        print(f"[WARN] {t}: no data");
        continue
    df.attrs['Ticker']=t
    df = add_indicators(df, FAST_DMA, SLOW_DMA, REGIME_DMA, RSI_LEN, ADX_LEN, ATR_LEN:=14)
    df = mark_crosses(df, FAST_DMA, SLOW_DMA)

    ent = entry_signal(
        df, FAST_DMA, SLOW_DMA,
        REQUIRE_CROSS_EXACTLY_T_MINUS_2, CROSS_WINDOW_AFTER,
        USE_FILTER_PRICE_ABOVE_200DMA, USE_FILTER_SLOW_SLOPE_POSITIVE,
        USE_FILTER_RSI, RSI_MIN,
        USE_FILTER_ADX, ADX_MIN,
        USE_FILTER_VOLUME_SURGE, VOL_LOOKBACK, VOL_FACTOR,
        USE_FILTER_ATR_BAND, ATR_MIN_PCT, ATR_MAX_PCT
    )
    exi = exit_signal(df, SLOW_DMA, PROTECTIVE_EXIT_ON_CLOSE_BELOW_20DMA)

    trades, ec = backtest_long_only(df, ent, exi, INITIAL_CAPITAL, RISK_PER_TRADE, SLIPPAGE_BPS, FEES_BPS)
    all_ec[t]=ec
    if not trades.empty: all_trades.append(trades.assign(Ticker=t))

    final_eq = ec['Equity'].iloc[-1]
    total_ret = final_eq/INITIAL_CAPITAL - 1.0
    dd = (ec['Equity']/ec['Equity'].cummax() - 1.0).min()
    perf_rows.append({'Ticker':t,'Trades':0 if trades.empty else len(trades)//2,
                      'FinalEquity':final_eq,'TotalReturn':total_ret,'MaxDrawdown':dd})

    # plot only if enabled
    if PLOT_LAST_N and PLOT_LAST_N>0:
        from datetime import datetime
        last_n = df.tail(PLOT_LAST_N).copy()
        # (plotting code omitted here to keep v2 lean; reuse from v1 if needed)

trades_df = pd.concat(all_trades, ignore_index=True) if all_trades else pd.DataFrame(
    columns=['Time','Ticker','Action','Qty','Price','Fees','PnL']
)
perf_df = pd.DataFrame(perf_rows).sort_values('TotalReturn', ascending=False)

trades_df.to_csv("dma_two_green_trades_v2.csv", index=False)
perf_df.to_csv("dma_two_green_performance_v2.csv", index=False)

print("\n=== Summary (v2) ===")
print(perf_df.to_string(index=False))
print("\nSaved: dma_two_green_trades_v2.csv, dma_two_green_performance_v2.csv")


Processing ATUL.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing CENTRALBK.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing COCHINSHIP.NSFACT.NS...


...

/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing TARIL.NS...


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


Processing UCOBANK.NS...

=== Summary (v2) ===
      Ticker  Trades  FinalEquity  TotalReturn  MaxDrawdown
     GSPL.NS       1 55555.930948     0.111119    -0.062789
  UCOBANK.NS       1 50027.427359     0.000549    -0.016630
CENTRALBK.NS       0 50000.000000     0.000000     0.000000
HINDPETRO.NS       0 50000.000000     0.000000     0.000000
     IRFC.NS       0 50000.000000     0.000000     0.000000
    LUPIN.NS       0 50000.000000     0.000000     0.000000
     NBCC.NS       0 50000.000000     0.000000     0.000000
    TARIL.NS       1 45948.448110    -0.081031    -0.100912
     ATUL.NS       3 42204.610631    -0.155908    -0.211267

Saved: dma_two_green_trades_v2.csv, dma_two_green_performance_v2.csv


/var/folders/mn/tx231v5158797chnp22wm9000000gp/T/ipykernel_14784/292461430.py:89: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cross_cond = o['CrossUp'].shift(2).fillna(False)


## Suggested Defaults to Reduce Losing Trades
- Keep **USE_FILTER_PRICE_ABOVE_200DMA = True**
- Keep **USE_FILTER_SLOW_SLOPE_POSITIVE = True**
- Use **RSI_MIN = 50–55**
- Use **ADX_MIN = 18–25** depending on trendiness
- Enable **ATR band**; try `ATR_MIN_PCT=0.5` and `ATR_MAX_PCT=4.0`
- Consider enabling **volume surge** with `VOL_FACTOR=1.2–1.5`
- Keep **protective exit** on to avoid waiting for two red closes
